In [1]:
import json
from pathlib import Path
import logging
from datetime import datetime

import pandas as pd

## Configure Logging

In [2]:
# Create logs directory if it doesn't exist.
log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)

# Create log file with dynamic date-based name.
log_date = datetime.now().strftime("%Y%m%d")
filename_no_ext = Path().resolve().name
log_file = log_dir / f"{log_date}-{filename_no_ext}.log"

# Create named logger
LOGGER = logging.getLogger(filename_no_ext)
LOGGER.setLevel(logging.INFO)

# File handler
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))

# Optional console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(
    logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
)

# Add handlers to logger (avoid duplicates)
if not LOGGER.hasHandlers():
    LOGGER.addHandler(file_handler)
    LOGGER.addHandler(console_handler)

## Load & Sanitize Data
- Import the CSV, then sanitize the column names.
- Adjust `csv_inpath` as appropriate.
- Rename columns for clarity.

In [3]:
# Load the CSV-formatted use cases from a file.
csv_inpath = Path("data/input/20250516-autodesk-jira-tickets-updated-sheet.csv")
df = pd.read_csv(
    csv_inpath, skiprows=1, usecols=[0, 1, 3, 4, 5, 6, 7, 8]
)  # Adding Status per customer request.
renames = {
    "Use Case Title": "use_case_title",
    "Product": "product",
    "ticket number": "ticket_number",
    "jira ticket": "jira_ticket",
    "Prompts": "prompts",
    "Purpose of Use": "purpose_of_use",
    "Response to Request": "response_to_request",
    "Status": "status",
}
df.rename(columns=renames, inplace=True)
df.head()  # Uncomment to check input. Be sure not to share sensitive data.

,use_case_title,product,ticket_number,jira_ticket,status,prompts,purpose_of_use,response_to_request
0,Image resolution for Visual Production Artists,Super Resolution Generative Adversarial Networ...,87,https://jira.autodesk.com/browse/GENAIGOV-87,Approved,customer-facing features' thumbnailsexamples: ...,While creating a feature pages for our Autodes...,Please work with your LBP to ensure that the a...
1,Creating Content Clusters for Industry Specifi...,Abacus Agent AI and ChatLLM,144,https://jira.autodesk.com/browse/GENAIGOV-144,Approved,persona based prompts: Generate an Outreach em...,create content clusters - industry specific co...,Your request to use Abacus Agent AI and ChatLL...
2,Developing learning materials,Absorb LMS - Absorb Create,201,https://jira.autodesk.com/browse/GENAIGOV-201,Approved,"text descriptions, documentation",developing learning materials,Your request to use the Absorb AI content gene...
3,"image and text generation in photoshop, illust...",Adobe Firefly,157,https://jira.autodesk.com/browse/GENAIGOV-157,Approved,Generative AI image and text creations in Adob...,"image and text generation in photoshop, illust...","The described use case is approved; provided, ..."
4,increase image resolution (tool uses GAN),AI Image Upscaler,155,https://jira.autodesk.com/browse/GENAIGOV-155,Approved,Software screenshots or video stills that are ...,increase image resolution (tool uses GAN),"The described use case is approved; provided, ..."


## Execution
- Iterate down the rows of the CSV, transforming each one into a JSON object defined by [`use-case-schema.json`](./use-case-schema.json).

In [4]:
# status_mapping = {"Approved": 1, "Pending": 0, "Rejected": -1}
use_cases = []
# now_iso = datetime.now(timezone.utc).isoformat()

# Row by row, transform each CSV-formatted use case into a JSON-formatted dict.
for i, row in df.iterrows():
    use_case = {
        # "id": str(uuid.uuid4()),
        "name": row["use_case_title"].strip(),
        "description": (
            f"Purpose of Use:\n{row['purpose_of_use']}\n\n"
            f"Response to Request:\n{row['response_to_request']}\n\n"
            f"Prompts:\n{row['prompts']}\n\n"
            f"Product:\n{row['prompts']}\n\n"
            f"Ticket Number: {row['ticket_number']}\n"
            f"JIRA Ticket: {row['jira_ticket']}\n\n"
            f"Status: {row['status']}\n"
        ),
        # "icon": None,
        # "ai_type": "",
        # "governance_status": status_mapping.get(row["status"].strip(), 0),
        # "domains": [],
        # "industries": [],
        # "regions": [],
        # "risk_category_level": None,
        # "custom_fields": [],
        # "questionnaires": [],
        # "inserted_at": now_iso,
        # "updated_at": now_iso,
    }
    desc_len = len(use_case["description"])
    row_field = f"Row {i + 2}"
    len_field = f"({desc_len})"
    if desc_len > 255:
        LOGGER.warning(
            "Description field too long: "
            f"(in Excel file) {row_field:<7s}|{len_field:>7s} | {use_case['name']}"
        )
    use_cases.append(use_case)

# Save the list of JSON-formatted (dict) use cases to a file.
outfolder = csv_inpath.parents[1] / "output"
outfolder.mkdir(parents=True, exist_ok=True)
filename = csv_inpath.with_name(csv_inpath.stem + "-bulk-upload.json").name
json_outpath = outfolder / filename
payload = {"data": {"items": use_cases}}
with open(json_outpath, "w") as f:
    json.dump(payload, f, indent=4)

2025-05-20 11:54:58,110 [WARNING] Description field too long: (in Excel file) Row 2  | (2763) | Image resolution for Visual Production Artists
2025-05-20 11:54:58,111 [WARNING] Description field too long: (in Excel file) Row 3  | (1145) | Creating Content Clusters for Industry Specific Content
2025-05-20 11:54:58,111 [WARNING] Description field too long: (in Excel file) Row 4  |  (951) | Developing learning materials
2025-05-20 11:54:58,112 [WARNING] Description field too long: (in Excel file) Row 5  |  (762) | image and text generation in photoshop, illustrator, express and adobe stock
2025-05-20 11:54:58,113 [WARNING] Description field too long: (in Excel file) Row 6  |  (790) | increase image resolution (tool uses GAN)
2025-05-20 11:54:58,113 [WARNING] Description field too long: (in Excel file) Row 7  |  (681) | sentiment analysis on feedback, writing airtable formulas
2025-05-20 11:54:58,114 [WARNING] Description field too long: (in Excel file) Row 8  | (1066) | improve the effici